<a href="https://colab.research.google.com/github/AnnSenina/Python_DH_2023/blob/main/notebooks/Python_9_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8_%D0%B4%D0%BB%D1%8F_DH_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Базовая статистика

In [ ]:
# таблички
import pandas as pd

# статистика
import numpy as np
import scipy.stats as stats
from scipy.stats.contingency import association

# графика
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/AnnSenina/Python_DH_2023/main/data/BikeDataVar.csv')
df

In [ ]:
# быстрая базовая статистика
df.describe()

In [ ]:
# диаграмма, построенная с помощью pandas
# bins - количество столбиков
df['Humidity'].hist(bins=50);

In [ ]:
# как найти выбросы:
# обычно в исследованиях договариваются, что считать выбросами
# стандартный путь - вычислить межквартильный размах (3 квартиль - 1 квартиль)
# затем от этих квартилей заложить расстояние в 1.5 межквартильных размаха
d = df.describe()['Rental Count']['75%'] - df.describe()['Rental Count']['25%'] # межквартильный размах

print(len(df[df['Rental Count'] > df.describe()['Rental Count']['75%'] + 1.5 * d]), 'выбросов справа')
print(len(df[df['Rental Count'] < df.describe()['Rental Count']['25%'] - 1.5 * d]), 'выбросов слева')

In [ ]:
plt.boxplot(df['Rental Count']);

In [ ]:
d = df.describe()['Humidity']['75%'] - df.describe()['Humidity']['25%'] # межквартильный размах

print(len(df[df['Humidity'] > df.describe()['Humidity']['75%'] + 1.5 * d]), 'выбросов справа')
print(len(df[df['Humidity'] < df.describe()['Humidity']['25%'] - 1.5 * d]), 'выбросов слева')

In [ ]:
plt.boxplot(df['Humidity']);

In [ ]:
# данные в pandas можно группировать
df.groupby('Temperature Category')['Temperature'].median()

### сложности с модой...

Для сгруппированных данных нельзя напрямую посчитать моду

Но! Есть специальный метод .agg в pandas (агрегирование данных), который позволяет посчитать все, что угодно :)

Документация [здесь](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)

А вот здесь есть еще одна [тетрадка с примерами](https://dfedorov.spb.ru/pandas/%D0%9E%D0%B1%D1%8A%D1%8F%D1%81%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B9%20Grouper%20%D0%B8%20Agg%20%D0%B2%20Pandas.html) 

In [ ]:
df.groupby('Temperature Category')['Temperature'].agg(pd.Series.mode)

In [ ]:
df.groupby('Seasons')['Temperature Category'].agg(pd.Series.mode)
# даже моду к категориальным данным, сгруппированным по другой категориальной переменной :)

# Корреляционный анализ

Поиск зависимостей

In [ ]:
df.corr() # метод, встроенный в pandas
# работает для всех числовых значений датафрейма

In [ ]:
# обычно фильтруют

df[['Temperature',	'Humidity',	'Wind speed',	'Rainfall',	'Snowfall']].corr() 

In [ ]:
# очень распространенная визуализация - тепловая карта
# https://seaborn.pydata.org/generated/seaborn.heatmap.html
sns.heatmap(df[['Temperature',	'Humidity',	'Wind speed',	'Rainfall',	'Snowfall']].corr());

In [ ]:
# давайте ее еще немного настроим
sns.heatmap(df[['Temperature',	'Humidity',	'Wind speed',	'Rainfall',	'Snowfall']].corr(), cmap='RdYlGn', center=0, annot=True)
plt.title('Correlogram', fontsize=22);

### таблицы сопряженности

Для работы с категориальными данными придется поступать по-другому

In [ ]:
df_cat = pd.crosstab(df['Seasons'], df['Temperature Category']) # пока без промежуточных и итоговых сумм
df_cat

In [ ]:
stats.chi2_contingency(df_cat)

### Что это и как читать результат? 

Категориальные данные представлять числами некорректно - но для них можно составить частотную таблицу.

Далее будем проверять критерий хи-квадрат независимости Пирсона (внимание! критериев хи-квадрат - несколько, а основанных на них метрик - еще больше (тоный тест Фишера, коэффициент корреляции Крамера и др.)

Когда работаем с критерием хи-квадрат независимости, общая идея такая:

* нулевая гипотеза: переменные независимы друг от друга

* альтернативная гипотеза: между переменными етсь связь

* если pvalue <= 0.05, значит, нулевая гипотеза отвергается, переменные связаны

* если pvalue > 0.05, отвегрнуть нулевую гипотезу нельзя, переменные независимы друг от друга

In [ ]:
def vcramer(table):
    chi, p, _, _ = stats.chi2_contingency(table,correction=False)
    n = table.sum()
    r,c = table.shape
    return np.sqrt(chi/(n*(min(r,c)-1.))), p

vcramer(df_cat.values) 
# Крамер - коэффициент корреляции для категориальных данных
# Правда идеальный вариант - когда его считают к таблице 2 на 2, так результат точнее